### Semantic Search with FAISS

In [2]:
# load dataset
from datasets import load_dataset


df_install = load_dataset("carlosug/ResearchInstall", split="train")
df_install



Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['software', 'repo_name', 'readme_url', 'content', 'plan', 'steps', 'optional_steps', 'extra_info_optional'],
    num_rows: 39
})

In [3]:
columns = df_install.column_names
columns
columns_to_keep = ["software", "content", "readme_url", "steps", "plan"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
df_install = df_install.remove_columns(columns_to_remove)
df_install

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 39
})

In [4]:
df_install.to_pandas()

,software,readme_url,content,plan,steps
0,vcr-video-representation-for-contextual,https://raw.githubusercontent.com/oronnir/VCR/...,None,None,None
1,ensuring-trustworthy-and-ethical-behaviour-in,https://raw.githubusercontent.com/AAAI-DISIM-U...,## Installation\n\n**OS X & Linux:**\n1. To d...,"binary, source",[plan binary]>>step1. follow the instructions ...
2,synthesizing-sentiment-controlled-feedback,https://raw.githubusercontent.com/MIntelligenc...,None,None,None
3,only-the-curve-shape-matters-training,https://raw.githubusercontent.com/cfeng783/GTT...,## Getting Started\n\n#### Install dependencie...,source,[plan source]>> [INCOMPLETE] step1. Install de...
4,from-uncertainty-to-precision-enhancing,https://raw.githubusercontent.com/fer-agathe/c...,None,None,None
5,stochastic-gradient-flow-dynamics-of-test,https://raw.githubusercontent.com/rodsveiga/sg...,None,None,None
6,accuracy-of-textfooler-black-box-adversarial,https://raw.githubusercontent.com/zero-one-los...,None,None,None
7,differentially-private-decentralized-learning-1,https://raw.githubusercontent.com/totilas/DPra...,None,None,None
8,aydiv-adaptable-yielding-3d-object-detection,https://raw.githubusercontent.com/sanjay-810/A...,### **Installation**\n1. Prepare for the runn...,"source,docker",[plan source]>> step1. Prepare for the running...
9,cartesian-atomic-cluster-expansion-for,https://raw.githubusercontent.com/BingqingChen...,## Installation\n\nPlease refer to the `setup....,source,[plan source]>>[INCOMPLETE] step1. please refe...


In [5]:
df_install = df_install.filter(
    lambda x: (x["content"] != None)
)
df_install

Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 29
})

In [6]:
df_install.set_format("pandas")
df2install = df_install[:]
# df2install["readme_url"][0].tolist() error str object has no attribute 'tolist'

In [7]:
df_install.set_format("pandas")
dataset_carlos = df_install[:]
dataset_carlos["steps"][2]

'[plan source]>> step1. Prepare for the running environment. step2. prepare for the data:```cd Aydiv python depth_to_lidar.py ```\n[plan docker]>> step1. You can use the docker image provided by [`OpenPCDet`](https://github.com/open-mmlab/OpenPCDet)'

In [26]:
content_df = df2install.explode("content", ignore_index=True)
content_df.head(4)
content_df.dtypes

software      object
readme_url    object
content       object
plan          object
steps         object
dtype: object

In [20]:
from datasets import Dataset

content_dataset = Dataset.from_pandas(content_df)
content_dataset

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps'],
    num_rows: 29
})

In [21]:
content_dataset = content_dataset.map(
    lambda x: {"content_length": len(x["content"].split())} # to count number of rows in a cell comments
)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [22]:
# content_dataset = content_dataset.filter(lambda x: x["content_length"] > 1) # filter small comments
# content_dataset

In [28]:
def concatenate_text(examples):
    return {
        "text": examples["software"]
        + " \n "
        + examples["content"]
        + " \n "
        + examples["plan"]
        + " \n "
        + examples["steps"]
        + " \n "
        # + examples["seq_order"] no string
    }


content_dataset = content_dataset.map(concatenate_text)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [29]:
content_dataset["text"]



['ensuring-trustworthy-and-ethical-behaviour-in \n ## Installation\n\n**OS X & Linux:**\n1.  To download and install SICStus Prolog (it is needed), follow the instructions at https://sicstus.sics.se/download4.html.\n2.  Then, you can download DALI and test it by running an example DALI MAS:\n```sh\ngit clone https://github.com/AAAI-DISIM-UnivAQ/DALI.git\ncd DALI/Examples/advanced\nbash startmas.sh\n```\n&nbsp;&nbsp;&nbsp;&nbsp; You will see different windows opening:\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog LINDA server (active_server_wi.pl)\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog FIPA client (active_user_wi.pl) \n* &nbsp;&nbsp;&nbsp;&nbsp; 1 instance of DALI metaintepreter for each agent (active_dali_wi.pl)\n\n**Windows:**\n1.  To download and install SICStus Prolog (it is needed), follow the instructions at https://sicstus.sics.se/download4.html.\n2.  Then, you can download DALI from https://github.com/AAAI-DISIM-UnivAQ/DALI.git.\n3.  Unzip the repository, go to the folder "DALI/Examples/basic"

In [30]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [31]:
import torch

device = torch.device("cpu")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [32]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [33]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [34]:
# from the dataset
embedding = get_embeddings(content_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [35]:
embeddings_dataset = content_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
# !conda install faiss-cpu -y # algorithms to quickly search and cluster embedding vectors it needs to be installed via conda

In [ ]:
# !conda install -c pytorch faiss-cpu -y

In [156]:
# import faiss-cpu conda
# embeddings_dataset.add_faiss_index(column="embeddings")

100%|██████████| 3/3 [00:00<00:00, 340.74it/s]


Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [36]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['software', 'readme_url', 'content', 'plan', 'steps', 'content_length', 'text', 'embeddings'],
    num_rows: 29
})

In [44]:
question = 'what is the plan of https://raw.githubusercontent.com/cfeng783/GTT?'
question_embedding = get_embeddings([question]).cpu().detach().numpy()

question_embedding.shape

(1, 768)

In [45]:
scores, samples = embeddings_dataset.get_nearest_examples("embeddings", question_embedding, k=5)

In [46]:
import pandas as pd

samples_df= pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=False)

,software,readme_url,content,plan,steps,content_length,text,embeddings,scores
4,only-the-curve-shape-matters-training,https://raw.githubusercontent.com/cfeng783/GTT...,## Getting Started\n\n#### Install dependencie...,source,[plan source]>> [INCOMPLETE] step1. Install de...,15,only-the-curve-shape-matters-training \n ## Ge...,"[-0.25307658314704895, -0.6750497221946716, -0...",48.823574
3,genegpt-teaching-large-language-models-to-use,https://raw.githubusercontent.com/ncbi/GeneGPT...,# Requirements\n\nThe code has been tested wit...,source,[plan source]>>step1.install requirements:\n``...,104,genegpt-teaching-large-language-models-to-use ...,"[-0.4030008614063263, -0.32260626554489136, -0...",47.592957
2,geometric-slosh-free-tracking-for-robotic,https://raw.githubusercontent.com/jonarriza96/...,## Installation\n\n### Dependencies\n\nInitial...,source,[plan source]>> step1. Check dependencies. ste...,69,geometric-slosh-free-tracking-for-robotic \n #...,"[-0.3325411379337311, -0.45105645060539246, -0...",44.597107
1,guiding-instruction-based-image-editing-via,https://raw.githubusercontent.com/apple/ml-mgi...,## Requirements\n```\nconda create -n mgie pyt...,source,[plan source]>> step1. create conda environmen...,121,guiding-instruction-based-image-editing-via \n...,"[-0.19101731479167938, -0.4127528667449951, -0...",43.995201
0,tensorflow,https://raw.githubusercontent.com/tensorflow/t...,## Install\n\nSee the [TensorFlow install guid...,packagemanager,[plan packagemanager]>>via pip. step1.:\n```\n...,95,tensorflow \n ## Install\n\nSee the [TensorFlo...,"[-0.4453235864639282, -0.4245409369468689, -0....",43.752434


In [47]:
for _, row in samples_df.iterrows():
    print(f"CONTENT: {row.content}")
    print(f"SCORE: {row.scores}")
    print(f"SOFTWARE: {row.software}")
    print(f"METHOD: {row.plan}")
    print(f"STEPS: {row.steps}")
    print("=" * 50)
    print()

CONTENT: ## Install

See the [TensorFlow install guide](https://www.tensorflow.org/install) for the
[pip package](https://www.tensorflow.org/install/pip), to
[enable GPU support](https://www.tensorflow.org/install/gpu), use a
[Docker container](https://www.tensorflow.org/install/docker), and
[build from source](https://www.tensorflow.org/install/source).

To install the current release, which includes support for
[CUDA-enabled GPU cards](https://www.tensorflow.org/install/gpu) *(Ubuntu and
Windows)*:

```
$ pip install tensorflow
```

Other devices (DirectX and MacOS-metal) are supported using
[Device plugins](https://www.tensorflow.org/install/gpu_plugins#available_devices).

A smaller CPU-only package is also available:

```
$ pip install tensorflow-cpu
```

To update TensorFlow to the latest version, add `--upgrade` flag to the above
commands.

*Nightly binaries are available for testing using the
[tf-nightly](https://pypi.python.org/pypi/tf-nightly) and
[tf-nightly-cpu](https://pyp

---

## Fine tuning pretarained models from scratch (issue with length)
[https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter7/section6_pt.ipynb#scrollTo=YjEWemr39kLy](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter7/section6_pt.ipynb#scrollTo=YjEWemr39kLy)

[https://huggingface.co/learn/nlp-course/chapter7/6](https://huggingface.co/learn/nlp-course/chapter7/6)

In [33]:
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", torch_dtype=torch.bfloat16)
# model = model.to(device).eval()

In [15]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# # Same as before
# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# sequences = [
#     "I've been waiting for a HuggingFace course my whole life.",
#     "This course is amazing!",
# ]
# batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# # This is new
# batch["labels"] = torch.tensor([1, 1])

# optimizer = AdamW(model.parameters())
# loss = model(**batch).loss
# loss.backward()
# optimizer.step()

In [26]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [27]:
raw_datasets["train"][0]



{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [23]:
from datasets import load_dataset

raw_plan_dataset = load_dataset("carlosug/ReadmeInstallationPlan")
raw_plan_dataset

DatasetDict({
    train: Dataset({
        features: ['readme_url', 'level of complexity', 'topic', 'url', 'token', 'text', 'sentence'],
        num_rows: 190
    })
})

In [127]:
from datasets import load_dataset


raw_step_dataset = load_dataset("carlosug/ResearchInstall")
raw_step_dataset
raw_step_dataset = raw_step_dataset.filter(
    lambda x: (x["content"] != None)
)
raw_step_dataset

Filter: 100%|██████████| 30/30 [00:00<00:00, 4408.56 examples/s]


DatasetDict({
    train: Dataset({
        features: ['software', 'repo_name', 'readme_url', 'portal', 'stars', 'selection', 'categories', 'Unnamed: 7', 'date_collection', 'date_submission', 'content', 'method', 'steps', 'seq_order', 'optional_steps', 'optional_instructions', 'seq_order.1', 'extra_info_optional', 'available_method (1=source, 2=packagemanager, 3=container, 4=binary)'],
        num_rows: 17
    })
})

In [24]:
columns = raw_plan_dataset["train"].column_names
columns
columns_to_keep = ["readme_url", "text", "url"]
columns_to_keep
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
columns_to_remove
raw_plan_dataset["train"] = raw_plan_dataset["train"].remove_columns(columns_to_remove)
raw_plan_dataset["train"] 

Dataset({
    features: ['readme_url', 'url', 'text'],
    num_rows: 190
})

In [128]:
# steps
columns = raw_step_dataset["train"].column_names
columns
columns_to_keep = ["repo_name", "content", "method", "steps"]
columns_to_keep
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
columns_to_remove
raw_step_dataset["train"] = raw_step_dataset["train"].remove_columns(columns_to_remove)
raw_step_dataset["train"] 

Dataset({
    features: ['repo_name', 'content', 'method', 'steps'],
    num_rows: 17
})

In [43]:
raw_plan_dataset_clean = raw_plan_dataset["train"].train_test_split(train_size=0.8, seed=3)
raw_plan_dataset_clean["validation"] = raw_plan_dataset_clean.pop("test")
raw_plan_dataset_clean["test"] = raw_plan_dataset["train"]
raw_plan_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 152
    })
    validation: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 38
    })
    test: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 190
    })
})

In [131]:
raw_step_dataset_clean = raw_step_dataset["train"].train_test_split(train_size=0.8, seed=3)
raw_step_dataset_clean["validation"] = raw_step_dataset_clean.pop("test")
raw_step_dataset_clean["test"] = raw_step_dataset["train"]
raw_step_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'content', 'method', 'steps'],
        num_rows: 13
    })
    validation: Dataset({
        features: ['repo_name', 'content', 'method', 'steps'],
        num_rows: 4
    })
    test: Dataset({
        features: ['repo_name', 'content', 'method', 'steps'],
        num_rows: 17
    })
})

In [45]:
raw_plan_dataset_clean["train"]

Dataset({
    features: ['readme_url', 'url', 'text'],
    num_rows: 152
})

In [46]:
len(raw_plan_dataset_clean["validation"])

38

In [47]:
# train_dataset = raw_plan_dataset["train"]
raw_plan_dataset_clean["validation"][0]

{'readme_url': 'https://raw.githubusercontent.com/h2oai/h2ogpt/main/README.md',
 'url': 'https://github.com/h2oai/h2ogpt',
 'text': 'for AMD ROC, comment-out all except the correct ROC wheel\n   pip install -r reqs_optional/requirements_optional_gpt4all.txt\n\n   pip install -r reqs_optional/requirements_optional_langchain.urls.txt\n   '}

In [132]:
raw_step_dataset_clean["validation"][1]

{'repo_name': 'https://bio.tools/',
 'content': '# Instruction of reproducing results in ViralCC paper\nWe take the cow fecal datasets for example. The other two datasets were processed following the same procedure.\n\nScripts to process the intermediate data and plot figures are available in the folder [Scripts](https://github.com/dyxstat/Reproduce_ViralCC/tree/main/Scripts).\n\nSource data of Figure 2 and 3 in the main text and Figure S1 in the supplementary materials are provided in the folder [Source Data](https://github.com/dyxstat/Reproduce_ViralCC/tree/main/Source%20Data).\n\n**Version of softwares exploited in the analyses**\n```\nfastq_dump command from Sratoolkit: v2.10.8\n\nbbduk.sh and clumpify.sh command from BBTools suite: v37.25\n\nmegahit command from MEGAHIT: v1.2.9\n\nbwa command from BWA MEM: v0.7.17\n\nsamtools command from Samtools: v1.15.1\n\nwrapper_phage_contigs_sorter_iPlant.pl command from VirSorter: v1.0.6\n\ncheckv command from CheckV: 0.7.0\n```\n\n**Step 1

In [48]:
raw_plan_dataset_clean["train"].features

{'readme_url': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [49]:
for key in raw_plan_dataset_clean["train"][0]:
    print(f"{key.upper()}: {raw_plan_dataset_clean['train'][0][key][:200]}")

README_URL: https://raw.githubusercontent.com/bentoml/OpenLLM/main/README.md
URL: https://github.com/bentoml/OpenLLM
TEXT: Install OpenLLM

Install OpenLLM by using `pip` as follows:

```bash
pip install openllm
```

To verify the installation, run:

```bash
$ openllm -h

Usage: openllm [OPTIONS] COMMAND [ARGS]...

   ███


In [133]:
for key in raw_step_dataset_clean["train"][0]:
    print(f"{key.upper()}: {raw_step_dataset_clean['train'][0][key][:200]}")

REPO_NAME: https://bio.tools/
CONTENT: **INSTALLATION FOR VERSION 1.1.6**

NRN-EZ was built with PyInstaller 3.6, and requires the following languages and libraries:

�	Python 3.6.9 and higher (currently up to 3.10)

�	PyQt 5.10.1

�	PyQtG
METHOD: binary
STEPS: 1. install requirements:
Python 3.6.9 and higher (currently up to 3.10)
PyQt 5.10.1
PyQtGraph 0.11.0
2. for linux:download the Linux zip file and, from the command window, run a bash command for the i


In [141]:
def show_samples(raw_step_dataset_clean, num_samples=3, seed=42):
    sample = raw_step_dataset_clean["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Method: {example['method']}'")
        print(f"'>> Content: {example['content']}'")
        print(f">> Steps: {example['steps']}")


show_samples(raw_step_dataset_clean)


'>> Method: binary'
'>> Content: **INSTALLATION FOR VERSION 1.1.6**

NRN-EZ was built with PyInstaller 3.6, and requires the following languages and libraries:

�	Python 3.6.9 and higher (currently up to 3.10)

�	PyQt 5.10.1

�	PyQtGraph 0.11.0

Installation instructions for Linux (Ubuntu and Pop!_OS): download the Linux zip file and, from the command window, run a bash command for the install.sh file, in the corresponding installation folder. 

Installation instructions for Mac OS: download the Mac zip file and copy the NRN-EZ app to the Applications folder. 

Installation instructions for Windows: download the Win zip file and run the installation wizard.'
>> Steps: 1. install requirements:
Python 3.6.9 and higher (currently up to 3.10)
PyQt 5.10.1
PyQtGraph 0.11.0
2. for linux:download the Linux zip file and, from the command window, run a bash command for the install.sh file, in the corresponding installation folder. 
2. for Mac OS: download the Mac zip file and copy the NRN-EZ ap

In [145]:
# max_input_length = 51
# max_target_length = 30


# def preprocess_function(examples):
#     model_inputs = tokenizer(
#         examples["content"],
#         max_length=max_input_length,
#         truncation=True,
#     )
#     labels = tokenizer(
#         examples["method"], max_length=max_target_length, truncation=True
#     )
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [148]:
# tokenized_datasets = raw_step_dataset_clean.map(preprocess_function, batched=True)

In [149]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_step_dataset_clean["train"][:2]["content"],
    truncation=True,
    max_length=context_length, #
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 4
Input chunk lengths: [128, 40, 128, 5]
Chunk mapping: [0, 0, 1, 1]


In [18]:
# in case complex datataset
# flat_dataset = plan_dataset_train.flatten()
# flat_dataset.features

In [115]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer_sentence1 = tokenizer(raw_train_dataset[0]["sentence1"])
# tokenizer_sentence2 = tokenizer(raw_train_dataset[0]["sentence2"])

# # alternative option
# inputs = tokenizer("This is the first sentence.", "This is the second one.")
# inputs



In [150]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length= context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets_step = raw_step_dataset_clean.map(
    tokenize, batched=True, remove_columns=raw_step_dataset_clean["train"].column_names
)
tokenized_datasets_step

Map: 100%|██████████| 17/17 [00:00<00:00, 2236.75 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 17
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 22
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 39
    })
})

In [110]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_plan_dataset_clean["train"][:2]["text"],
    truncation=True,
    max_length=context_length, #
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 9
Input chunk lengths: [128, 128, 128, 128, 128, 12, 128, 128, 119]
Chunk mapping: [0, 0, 0, 0, 0, 0, 1, 1, 1]


In [80]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length= context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_plan_dataset_clean.map(
    tokenize, batched=True, remove_columns=raw_plan_dataset_clean["train"].column_names
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 254
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 57
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 311
    })
})

In [151]:
# metrics for summarization
generated_method = "You need to download the software via the following URL source" # machine generated
reference_method = "Source" # human true

In [152]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [172]:
# create baseline
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize


# create a simple function to extract the first three sentences in a installation. 

def three_installation(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_installation(raw_step_dataset_clean["train"][4]["content"]))

## Getting Started

#### Install dependencies (with python 3.10) 

```shell
pip install -r requirements.txt
```


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [165]:
def evaluate_baseline(dataset, metric):
    installations = [three_installation(text) for text in dataset["content"]]
    return metric.compute(predictions=installations, references=dataset["method"])

In [175]:
# import pandas as pd
# import evaluate
# rouge_score = evaluate.load("rouge")

# score = evaluate_baseline(raw_step_dataset_clean, rouge_score)
# rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
# rouge_dict -- PROBLEM

TypeError: expected string or bytes-like object

In [177]:
# Fine tuning m15 with Trainer API
from transformers import AutoModelForSeq2SeqLM

model_checkpoint = "google/mt5-small"

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin: 100%|██████████| 1.20G/1.20G [03:41<00:00, 5.42MB/s]
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 136kB/s]


In [178]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    # push_to_hub=True,
)

In [179]:
import numpy as np
import pandas as pd
import evaluate
rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [87]:
# Initiate a model
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [186]:
tokenized_datasets = tokenized_datasets.column_names

In [187]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [188]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

IndexError: list index out of range

In [82]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.2M parameters


In [83]:
# create batches
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [84]:
# check if vector all same shape
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [58]:
!pip install ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.10-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.2-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl (215 kB)
Using cached widgetsnbextension-4.0.10-py3-none-any.whl (2.3 MB)


In [59]:
from huggingface_hub import notebook_login

notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [88]:
# training arguments and fire up the Trainer
from transformers import Trainer, TrainingArguments

# args = TrainingArguments(
#     output_dir="codeparrot-ds",
#     # per_device_train_batch_size=32,
#     # per_device_eval_batch_size=32,
#     # evaluation_strategy="steps",
#     # eval_steps=5_000,
#     # logging_steps=5_000,
#     gradient_accumulation_steps=8,
#     num_train_epochs=1,
#     weight_decay=0.1,
#     warmup_steps=1_000,
#     lr_scheduler_type="cosine",
#     learning_rate=5e-4,
#     save_steps=5_000,
#     # fp16=False,
#     push_to_hub=False,
# )
args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    # eval_steps=5_000,
    # logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    weight_decay=0.011,
    # warmup_steps=1_000,
    # lr_scheduler_type="cosine",
    learning_rate=2e-5,
    save_steps=5_000,
    # fp16=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=raw_plan_dataset_clean["train"],
    eval_dataset=raw_plan_dataset_clean["validation"],
)

In [89]:
trainer.train() # issue with context_length

/Users/ccugutrillague/anaconda3/envs/fine-tune-conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


/Users/ccugutrillague/anaconda3/envs/fine-tune-conda/lib/python3.9/site-packages/datasets/formatting/formatting.py:100: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


IndexError: list index out of range

In [16]:
# from transformers import AutoTokenizer

# checkpoint = "huggingface-course/code-search-net-tokenizer"
# model_small = "google/mt5-small"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer_text = tokenizer(plan_dataset_train[0]["readme_url"])
# tokenizer_step = tokenizer(plan_dataset_train[0]["text"])

# # # alternative option
# # inputs = tokenizer("This is the first sentence.", "This is the second one.")
# # inputs


----
#  another

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_datasets["train"][0]



{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [91]:
from datasets import load_dataset


dataset = load_dataset("carlosug/ResearchInstall")
raw_plan_dataset

DatasetDict({
    train: Dataset({
        features: ['readme_url', 'level of complexity', 'topic', 'url', 'token', 'text', 'sentence'],
        num_rows: 190
    })
})

In [92]:
columns = raw_plan_dataset["train"].column_names
columns
columns_to_keep = ["readme_url", "text", "url"]
columns_to_keep
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
columns_to_remove
raw_plan_dataset["train"] = raw_plan_dataset["train"].remove_columns(columns_to_remove)
raw_plan_dataset["train"] 

Dataset({
    features: ['readme_url', 'url', 'text'],
    num_rows: 190
})

In [93]:
raw_plan_dataset_clean = raw_plan_dataset["train"].train_test_split(train_size=0.8, seed=3)
raw_plan_dataset_clean["validation"] = raw_plan_dataset_clean.pop("test")
raw_plan_dataset_clean["test"] = raw_plan_dataset["train"]
raw_plan_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 152
    })
    validation: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 38
    })
    test: Dataset({
        features: ['readme_url', 'url', 'text'],
        num_rows: 190
    })
})

In [94]:
raw_plan_dataset_clean["train"]

Dataset({
    features: ['readme_url', 'url', 'text'],
    num_rows: 152
})

In [95]:
len(raw_plan_dataset_clean["validation"])

38

In [96]:
# train_dataset = raw_plan_dataset["train"]
raw_plan_dataset_clean["validation"][0]

{'readme_url': 'https://raw.githubusercontent.com/h2oai/h2ogpt/main/README.md',
 'url': 'https://github.com/h2oai/h2ogpt',
 'text': 'for AMD ROC, comment-out all except the correct ROC wheel\n   pip install -r reqs_optional/requirements_optional_gpt4all.txt\n\n   pip install -r reqs_optional/requirements_optional_langchain.urls.txt\n   '}

In [97]:
raw_plan_dataset_clean["train"].features

{'readme_url': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [99]:
for key in raw_plan_dataset_clean["train"][0]:
    print(f"{key.upper()}: {raw_plan_dataset_clean['train'][3][key][:200]}")

README_URL: https://raw.githubusercontent.com/microsoft/promptflow/main/README.md
URL: https://github.com/microsoft/promptflow
TEXT: Quick Start ⚡

**Create a chatbot with prompt flow**

Run the command to initiate a prompt flow from a chat template, it creates folder named `my_chatbot` and generates required files within it:

```s


In [101]:
from transformers import AutoTokenizer

model_checkpoint = "huggingface-course/code-search-net-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [102]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [41, 414, 1433, 5366, 256, 865, 28482, 704, 17601, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
# tokenized_dataset = tokenizer(
#     plan_dataset["train"]["readme_url"],
#     # plan_dataset["train"]["task_step"],
#     plan_dataset["train"]["text"],
#     padding=True,
#     truncation=True,
# )



Using pad_token, but it is not set yet.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [46]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [252]:
def tokenize_function1(example):
    # return tokenizer(example["user_request"], example["method"], truncation=True)
    return tokenizer(example["method"], example["text"], truncation=True, padding=True)

In [253]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map: 100%|██████████| 1725/1725 [00:00<00:00, 24424.76 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [254]:
tokenized_datasets1 = plan_dataset.map(tokenize_function1, batched=True)
tokenized_datasets1

Map: 100%|██████████| 56/56 [00:00<00:00, 5295.00 examples/s]


DatasetDict({
    train: Dataset({
        features: ['method', 'step_number', 'label', 'task_step', 'text', 'url', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 56
    })
})

In [237]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [255]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "user_request", "id"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [256]:
samples = tokenized_datasets1["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "method", "text"]}
[len(x) for x in samples["input_ids"]]

[156, 156, 156, 156, 156, 156, 156, 156]

In [257]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [258]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [51]:
from transformers import TrainingArguments

# define Training arguments

training_args = TrainingArguments(
    "test_trainer",
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    # eval_accumulation_steps=1,
    # prediction_loss_only=True,
    # learning_rate=2e-5,
    # num_train_epochs=1,
    # weight_decay=0.01,
)

# define model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [52]:
# fefine the trainer
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [53]:
# fine tune the model with our dataset
trainer.train()

/Users/ccugutrillague/anaconda3/envs/fine-tune-conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 36%|███▋      | 500/1377 [06:07<10:50,  1.35it/s]

{'loss': 0.5336, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


 73%|███████▎  | 1000/1377 [12:39<05:00,  1.26it/s]

{'loss': 0.3153, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [17:48<00:00,  1.29it/s]

{'train_runtime': 1068.9181, 'train_samples_per_second': 10.295, 'train_steps_per_second': 1.288, 'train_loss': 0.3498013509557831, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3498013509557831, metrics={'train_runtime': 1068.9181, 'train_samples_per_second': 10.295, 'train_steps_per_second': 1.288, 'train_loss': 0.3498013509557831, 'epoch': 3.0})

In [54]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

100%|██████████| 51/51 [00:10<00:00,  4.98it/s]

(408, 2) (408,)


In [55]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [63]:
!conda install anaconda::scikit-learn -y
#  conda install conda-forge::scipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Channels:
 - bioconda
 - conda-forge
 - defaults
 - anaconda
 - pytorch
 - pytorch-nightly
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /Users/ccugutrillague/anaconda3/envs/fine-tune-conda

  added / updated specs:
    - anaconda::scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-1.3.0         |   py39h46d7db6_0         9.2 MB  anaconda
    scipy-1.12.0               |   py39hcc04109_2        14.0 MB  conda-forge
    threadpoolctl-3.3.0        |     pyhc1e730c_0          22 KB  conda-forge
    -----

In [64]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'accuracy': 0.8382352941176471, 'f1': 0.8888888888888888}

In [65]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [66]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [69]:
trainer.train()

 46%|████▌     | 636/1377 [4:47:05<5:34:29, 27.08s/it]
                                                  
 33%|███▎      | 459/1377 [05:52<10:09,  1.51it/s]

{'eval_loss': 0.6359043717384338, 'eval_accuracy': 0.875, 'eval_f1': 0.9128205128205128, 'eval_runtime': 12.2762, 'eval_samples_per_second': 33.235, 'eval_steps_per_second': 4.154, 'epoch': 1.0}


 36%|███▋      | 500/1377 [06:23<10:31,  1.39it/s]  

{'loss': 0.0517, 'learning_rate': 0.0, 'epoch': 1.09}


                                                  
 67%|██████▋   | 918/1377 [11:44<05:08,  1.49it/s]

...

 73%|███████▎  | 1000/1377 [12:45<04:35,  1.37it/s]

{'loss': 0.064, 'learning_rate': 0.0, 'epoch': 2.18}


                                                   
100%|██████████| 1377/1377 [17:46<00:00,  1.29it/s]

{'eval_loss': 0.6359043717384338, 'eval_accuracy': 0.875, 'eval_f1': 0.9128205128205128, 'eval_runtime': 12.2628, 'eval_samples_per_second': 33.271, 'eval_steps_per_second': 4.159, 'epoch': 3.0}
{'train_runtime': 1066.7174, 'train_samples_per_second': 10.316, 'train_steps_per_second': 1.291, 'train_loss': 0.05975601789132356, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.05975601789132356, metrics={'train_runtime': 1066.7174, 'train_samples_per_second': 10.316, 'train_steps_per_second': 1.291, 'train_loss': 0.05975601789132356, 'epoch': 3.0})